## Week 8
### Building a simple biosphere carbon box model in Python

(For Week 7 notes, see Prof Randerson's notes under modules 4 and 5. We looked at CMIP6 future projections and mapping with cartopy)

Euler's method approach

Cb(t) = Cb(t-1) + (NPP - Rh)

In [1]:
import numpy as np
import matplotlib.pyplot as py

In [17]:
filedata = np.loadtxt('ObsCO2.txt', skiprows=1, delimiter='\t')

In [18]:
date = filedata[:,0]
CO2obs = filedata[:,1]

In [21]:
print('start date', date[0], 'end date', date[-1])

start date 1765.0 end date 2005.0


#### Initialize the model

Assume that we started at steady state. In 1765, dC/dt = 0 = npp - rh

Cb = npp*tau

In [22]:
nyears = len(date)

Cb = np.zeros(nyears)
npp = np.zeros(nyears)
rh = np.zeros(nyears)

npp_initial = 50 #units PgC/yr
taub = 20 #units years, initial turnover time for biosphere
beta = 0.6 #sensitivity of NPP to CO2

Cb[0] = npp_initial*taub
rh[0] = npp_initial

In [24]:
for t in range(1, nyears):  #starting at 1766
    npp[t] = npp[0] * (1 + beta*np.log( CO2obs[t]/CO2obs[0] ))